In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from glob import iglob
import re
from datetime import datetime
import random
import cv2
from copy import copy
from importlib import reload
from glob import iglob
import os
import shutil

import sys
sys.path.append('..')
from src import utils as ut

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip install roboflow

from roboflow import Roboflow

API_KEY = "Ef3g07dPh87XxgZy6Vbw"
rf = Roboflow(api_key=API_KEY)

project = rf.workspace("ufsc-onwy0").project("lapix-roadobjsegmentation-v3")
version = project.version(4)
dataset = version.download("coco-segmentation")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 KB 981.7 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 KB 2.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 53.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 KB 45.2 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.12.1
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.2.2
    Uninstalling certifi-2024.2.2:
      Successfully uninstalled certifi-2024.2.2
loading Roboflow workspace...
l


Extracting Dataset Version Zip to lapix-RoadObjSegmentation-v3-4 in coco-segmentation:: 100%|████████████████████████████████████████████████████████████████████████████████████| 363/363 [00:00<00:00, 871.62it/s]


In [6]:
cat_ids
coco.cats

{0: {'id': 0, 'name': 'road-objects-P0qE-Ts2j', 'supercategory': 'none'},
 1: {'id': 1, 'name': 'Animals', 'supercategory': 'road-objects-P0qE-Ts2j'},
 2: {'id': 2, 'name': 'Asphalt', 'supercategory': 'road-objects-P0qE-Ts2j'},
 3: {'id': 3, 'name': 'Cat-s-Eye', 'supercategory': 'road-objects-P0qE-Ts2j'},
 4: {'id': 4, 'name': 'Cracks', 'supercategory': 'road-objects-P0qE-Ts2j'},
 5: {'id': 5, 'name': 'Ego', 'supercategory': 'road-objects-P0qE-Ts2j'},
 6: {'id': 6, 'name': 'Hard-Sand', 'supercategory': 'road-objects-P0qE-Ts2j'},
 7: {'id': 7, 'name': 'Markings', 'supercategory': 'road-objects-P0qE-Ts2j'},
 8: {'id': 8, 'name': 'Obstacle', 'supercategory': 'road-objects-P0qE-Ts2j'},
 9: {'id': 9, 'name': 'People', 'supercategory': 'road-objects-P0qE-Ts2j'},
 10: {'id': 10, 'name': 'Pothole', 'supercategory': 'road-objects-P0qE-Ts2j'},
 11: {'id': 11,
  'name': 'Retaining-wall',
  'supercategory': 'road-objects-P0qE-Ts2j'},
 12: {'id': 12,
  'name': 'Soft-Sand',
  'supercategory': 'road-

In [12]:
mocamba_classes = np.arange(17)
mocamba_classes[mocamba_classes == 15] = 0
mocamba_classes[mocamba_classes > 15] -= 1
mocamba_classes

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,  0, 15])

In [15]:
from pycocotools.coco import COCO

cocofiles = [f'lapix-RoadObjSegmentation-v3-4/{setname}/_annotations.coco.json' for setname in ['train', 'test']]

for cocofile in cocofiles:
    dirbase = os.path.dirname(cocofile)
    coco = COCO(cocofile)

    processor = ut.CocoProcessor(coco)
    cat_ids = coco.getCatIds()
    print(cat_ids)
    # break
    
    for ix in [*coco.imgs.keys()][:]:
        bname = coco.imgs[ix]['file_name']
        gt = processor.mask_from_cocoim(ix)
        
        p = os.path.join(dirbase, bname)
        setname = p.split('/')[-2]
        newp = os.path.join('ds-mocamba-v0.3.4', setname, 'im', os.path.basename(p))
        ut.create_dirpath_ifneeded(newp)
        shutil.copy(p, newp)
        
        newpgt = ut.get_gtpath(newp)
        ut.create_dirpath_ifneeded(newpgt)
        # remove vertical-sign class and re-index the other class
        gt = mocamba_classes[gt]
        cv2.imwrite(newpgt, gt)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


# Resize images

In [16]:
fpaths = list(iglob('ds-mocamba-v0.3.4/**/*.jpg', recursive=True))
print(len(fpaths))

357


In [20]:
long = 512*3
get_newpath = lambda p: p.replace('ds-mocamba-v0.3.4/', f'ds-mocamba-v0.3.4-long{long}/')

for p in fpaths[:]:    
    # if os.path.exists(newp):
    #     continue
    
    im = cv2.imread(p, cv2.IMREAD_UNCHANGED)
    newim = ut.resize_im(im, long=long)
    newp = get_newpath(p)
    ut.create_dirpath_ifneeded(newp)
    cv2.imwrite(newp, newim)

    pgt = ut.get_gtpath(p)
    gt = cv2.imread(pgt, cv2.IMREAD_UNCHANGED)
    newgt = ut.resize_im(gt, inter_nearest=True, long=long)
    newpgt = get_newpath(pgt)
    ut.create_dirpath_ifneeded(newpgt)
    cv2.imwrite(newpgt, newgt)

In [21]:
print("ACABOU!")

ACABOU!


In [21]:
fpaths = list(iglob('/home/rafael/workspace/PX-Matting/notebooks/cache/rtk-mocamba-experiments/data/mocamba/ds-mocamba-v0.3-long1024/train/im/*'))
print(len(fpaths))
# fpaths += list(iglob('/home/rafael/workspace/PX-Matting/notebooks/cache/rtk-mocamba-experiments/data/RTK/RTK-v1.1-long704/val/im/*'))
# print(len(fpaths))


txt = '\n'.join(fpaths) + '\n'
with open('/home/rafael/workspace/PX-Matting/notebooks/cache/rtk-mocamba-experiments/data/mocamba/ds-mocamba-v0.3-long1024/trainpaths.txt', 'w') as f:
    f.write(txt)

287
